# Chapter 2: Working with Text Data

Packages that are being used in this notebook:

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.8.0+cu126
tiktoken version: 0.11.0


- This chapter covers data preparation and sampling to get input data "ready" for the LLM

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="500px">

## 2.1 Understanding word embeddings

- There are many forms of embeddings; we focus on text embeddings in this book

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">

- LLMs work with embeddings in high-dimensional spaces (i.e., thousands of dimensions)
- Since we can't visualize such high-dimensional spaces (we humans think in 1, 2, or 3 dimensions), the figure below illustrates a 2-dimensional embedding space

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">

## 2.2 Tokenizing text

- In this section, we tokenize text, which means breaking text into smaller units, such as individual words and punctuation characters

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="300px">

- Load raw text we want to work with
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict) is a public domain short story

In [6]:
# Import the 'os' module to interact with the operating system, like checking for files.
import os

# Import the 'urllib.request' module to handle URLs and download files from the internet.
import urllib.request

# Check if the file "the-verdict.txt" does NOT already exist in the current directory.
# This prevents re-downloading the file every time the script is run.
if not os.path.exists("the-verdict.txt"):

    # Store the web address of the file we want to download in a variable called 'url'.
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")

    # Define the local filename for the downloaded file.
    file_path = "the-verdict.txt"

    # Download the file from the 'url' and save it to the local 'file_path'.
    urllib.request.urlretrieve(url, file_path)

In [7]:
# Open the file in read mode ("r") with UTF-8 encoding.
# The 'with' statement ensures the file is automatically closed afterward.
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    # Read the entire content of the file into a single string variable.
    raw_text = f.read()

# Print the total number of characters by getting the length of the string.
print("Total number of characters:", len(raw_text))

# Print a preview of the text: the first 99 characters.
print(raw_text[:99])

Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- The goal is to tokenize and embed this text for an LLM
- Let's develop a simple tokenizer based on some simple sample text that we can then later apply to the text above
- The following regular expression will split on whitespaces

In [9]:
# Import the 're' module, which provides support for regular expressions in Python.
import re

# Define a sample string that we will split.
text = "Hello, world. This, is a test."

# Use the re.split() function to break the string apart.
# r'(\s)' is the regular expression pattern:
#   - \s matches any whitespace character (like a space, tab, or newline).
#   - The parentheses () around \s create a "capturing group". This tells
#     re.split() to not only split by the whitespace but to also keep
#     the whitespace characters in the resulting list.
result = re.split(r'(\s)', text)

# Print the final list.
# The output will show the words and the whitespace characters that separated them.
print(result)

# Expected output:
# ['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- We don't only want to split on whitespaces but also commas and periods, so let's modify the regular expression to do that as well

In [10]:
# Assuming 're' is already imported and 'text' is defined from the previous example:
# import re
# text = "Hello, world. This, is a test."

# Use re.split() again, but with a more complex pattern.
# The pattern r'([,.]|\s)' means:
#   - The square brackets `[]` create a character set. `[,.]` matches a single
#     character that is either a comma ',' or a period '.'.
#   - The pipe `|` acts as an "OR".
#   - `\s` matches any whitespace character.
#   - So, `[,.]|\s` means "match a comma, OR a period, OR a whitespace".
#   - The outer parentheses `()` capture the matched delimiter, keeping it in the list.
result = re.split(r'([,.]|\s)', text)

# Print the resulting list.
# Notice that this pattern creates empty strings '' where two delimiters are
# next to each other (e.g., the comma and the space in "world. This,").
print(result)

# Expected output:
# ['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.']

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- As we can see, this creates empty strings, let's remove them

In [11]:
# Assume 'result' is the list from the previous step:
# ['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.']

# This line uses a list comprehension to create a new, filtered list.
# Let's break it down:
# 1. It iterates through each `item` in the current `result` list.
# 2. For each `item`, it calls `item.strip()`. This method removes any
#    leading or trailing whitespace. For example, ' ' becomes ''.
# 3. The `if` statement then checks if the result of `item.strip()` is "truthy".
#    In Python, empty strings ('') are treated as `False`, while any non-empty
#    string is `True`.
# 4. Only the original `item`s that are not empty or just whitespace are
#    kept and included in the new list assigned back to the `result` variable.
result = [item.strip() for item in result if item.strip()]

# Print the final, cleaned-up list.
print(result)

# Expected output:
# ['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- This looks pretty good, but let's also handle other types of punctuation, such as periods, question marks, and so on

In [12]:
# Import the 're' module for regular expression operations.
import re

# Define a new sample string with more complex punctuation, including "--" and "?".
text = "Hello, world. Is this-- a test?"

# Use re.split() with an expanded pattern to tokenize the string.
# The pattern r'([,.:;?_!"()\']|--|\s)' has three alternatives, separated by the OR `|` symbol:
#  1. `[,.:;?_!"()\']`: This is a character set that matches any single punctuation
#     mark within the square brackets.
#  2. `--`: This matches the literal double-hyphen "--" as a single unit.
#  3. `\s`: This matches any whitespace character.
# The outer parentheses `()` form a capturing group, which ensures that these
# delimiters are kept in the list after splitting.
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)

# This list comprehension is the same as before. It serves as a cleanup step.
# It iterates through the list produced by re.split() and removes any empty
# strings ('') or strings containing only whitespace (' '), which can occur
# when multiple delimiters are adjacent.
result = [item.strip() for item in result if item.strip()]

# Print the final, cleaned-up list of tokens.
print(result)

# Expected output:
# ['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- This is pretty good, and we are now ready to apply this tokenization to the raw text

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">

In [13]:
# Assume 're' is imported and 'raw_text' contains the full text from "the-verdict.txt".

# 1. Tokenization:
# This line applies the complex regular expression to the entire `raw_text`.
# It splits the text by various punctuation marks, the double-hyphen, and whitespace.
# The capturing group `()` ensures all these words and delimiters are kept,
# resulting in a very long list of tokens. The list is stored in a variable
# named `preprocessed`, indicating it's an early step in text processing.
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)

# 2. Cleanup:
# This is the same list comprehension used before. It iterates through the long
# `preprocessed` list and removes all the "noise" items—the empty strings ('')
# and whitespace-only strings (' ')—that were generated by the split.
# The result is a clean list of words and punctuation marks.
preprocessed = [item.strip() for item in preprocessed if item.strip()]

# 3. Preview the result:
# The `preprocessed` list now contains thousands of tokens from the entire story.
# Printing the whole list would be overwhelming.
# `preprocessed[:30]` uses list slicing to get only the first 30 items
# from the list, providing a manageable preview to verify that our
# tokenization and cleanup steps worked correctly.
print(preprocessed[:30])

# Expected output (showing the first 30 tokens from the story):
# ['I', 'The', 'Verdict', 'by', 'Anton', 'Chekhov', 'ON', 'an', 'autumn',
# 'evening', 'a', 'man', 'of', 'note', ',', 'whose', 'name', 'I', 'need',
# 'not', 'mention', 'here', ',', 'was', 'driving', 'from', 'N', '.', 'station', 'to']

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- Let's calculate the total number of tokens

In [15]:
# The `len()` function is a built-in Python function that returns the number
# of items in an object. When used on a list, it returns the total count of
# elements in that list.
# In this context, it's counting every word and punctuation mark that was
# separated and cleaned in the previous steps.
print(len(preprocessed))

4690


## 2.3 Converting tokens into token IDs

- Next, we convert the text tokens into token IDs that we can process via embedding layers later

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">

- From these tokens, we can now build a vocabulary that consists of all the unique tokens

In [16]:
# Create a sorted list of all unique tokens (the vocabulary).
# The `set()` function automatically removes all duplicates.
all_words = sorted(set(preprocessed))

# Count the total number of unique tokens to get the vocabulary size.
vocab_size = len(all_words)

# Print the vocabulary size.
print(vocab_size)

1130


In [17]:
# Create a vocabulary mapping from token to integer.
# This is a dictionary comprehension that iterates through each unique word.
# `enumerate()` gets both the index (integer) and the item (token).
# The result is a dictionary like {'a': 0, 'an': 1, 'and': 2, ...}
vocab = {token:integer for integer,token in enumerate(all_words)}

- Below are the first 50 entries in this vocabulary:

In [18]:
# Loop through the vocabulary dictionary's items.
# `enumerate` adds an index `i` (0, 1, 2, ...).
# `item` will be the (token, integer) pair from the dictionary.
for i, item in enumerate(vocab.items()):
    # Print the (token, integer) pair.
    print(item)

    # After printing the 51st item (at index 50), stop the loop.
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- Below, we illustrate the tokenization of a short sample text using a small vocabulary:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="500px">

- Putting it now all together into a tokenizer class

In [19]:
# We need the 're' module for our tokenization logic
import re

class SimpleTokenizerV1:
    """A simple tokenizer that performs string-to-integer and integer-to-string conversions."""

    def __init__(self, vocab):
        """
        Initializes the tokenizer with a vocabulary.
        Args:
            vocab (dict): A dictionary mapping strings to integers.
        """
        # Store the original string-to-integer mapping
        self.str_to_int = vocab

        # Create a reverse integer-to-string mapping for decoding
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        """Converts a text string into a list of integer IDs."""

        # 1. Tokenize the text using the same regex as before
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        # 2. Clean up by removing empty and whitespace-only tokens
        preprocessed = [item.strip() for item in preprocessed if item.strip()]

        # 3. Convert the cleaned tokens into their corresponding integer IDs
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        """Converts a list of integer IDs back into a text string."""

        # 1. Convert integer IDs back to string tokens and join with spaces
        text = " ".join([self.int_to_str[i] for i in ids])

        # 2. Use regex to remove spaces before certain punctuation marks for better readability
        # For example, converts "Hello , world ." to "Hello, world."
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- The `encode` function turns text into token IDs
- The `decode` function turns token IDs back into text

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">

- We can use the tokenizer to encode (that is, tokenize) texts into integers
- These integers can then be embedded (later) as input of/for the LLM

In [20]:
# 1. Create an instance of our tokenizer class.
#    We pass the `vocab` dictionary (the string-to-integer map)
#    to the tokenizer when we create it.
tokenizer = SimpleTokenizerV1(vocab)

# 2. Define a new text string that we want to encode.
#    The triple quotes `"""` allow the string to span multiple lines.
text = """"It's the last he painted, you know,"
          Mrs. Gisburn said with pardonable pride."""

# 3. Use the tokenizer's `encode` method to convert the text into a list of integer IDs.
ids = tokenizer.encode(text)

# 4. Print the resulting list of integer IDs.
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- We can decode the integers back into text

In [21]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [22]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 Adding special context tokens

- It's useful to add some "special" tokens for unknown words and to denote the end of a text

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">

- Some tokenizers use special tokens to help the LLM with additional context
- Some of these special tokens are
  - `[BOS]` (beginning of sequence) marks the beginning of text
  - `[EOS]` (end of sequence) marks where the text ends (this is usually used to concatenate multiple unrelated texts, e.g., two different Wikipedia articles or two different books, and so on)
  - `[PAD]` (padding) if we train LLMs with a batch size greater than 1 (we may include multiple texts with different lengths; with the padding token we pad the shorter texts to the longest length so that all texts have an equal length)
- `[UNK]` to represent words that are not included in the vocabulary

- Note that GPT-2 does not need any of these tokens mentioned above but only uses an `<|endoftext|>` token to reduce complexity
- The `<|endoftext|>` is analogous to the `[EOS]` token mentioned above
- GPT also uses the `<|endoftext|>` for padding (since we typically use a mask when training on batched inputs, we would not attend padded tokens anyways, so it does not matter what these tokens are)
- GPT-2 does not use an `<UNK>` token for out-of-vocabulary words; instead, GPT-2 uses a byte-pair encoding (BPE) tokenizer, which breaks down words into subword units which we will discuss in a later section



- We use the `<|endoftext|>` tokens between two independent sources of text:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">

- Let's see what happens if we tokenize the following text:

In [23]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

- The above produces an error because the word "Hello" is not contained in the vocabulary
- To deal with such cases, we can add special tokens like `"<|unk|>"` to the vocabulary to represent unknown words
- Since we are already extending the vocabulary, let's add another token called `"<|endoftext|>"` which is used in GPT-2 training to denote the end of a text (and it's also used between concatenated text, like if our training datasets consists of multiple articles, books, etc.)

In [24]:
# Re-create the sorted list of unique tokens from the source text.
all_tokens = sorted(list(set(preprocessed)))

# Add special tokens to the end of the vocabulary list.
# `<|endoftext|>` signals the end of a document.
# `<|unk|>` represents unknown words not in the vocabulary.
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

# Re-create the vocabulary dictionary, which now includes the special tokens.
# The special tokens will have the highest integer IDs since they are at the end of the list.
vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [25]:
len(vocab.items())

1132

In [26]:
# Inspect the last 5 items of the vocabulary to see our new special tokens.
# The `[-5:]` slice gets the final five elements from the list of vocab items.
for i, item in enumerate(list(vocab.items())[-5:]):
    # Print each (token, integer) pair.
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- We also need to adjust the tokenizer accordingly so that it knows when and how to use the new `<unk>` token

In [27]:
class SimpleTokenizerV2:
    # A robust tokenizer that handles unknown words by replacing them with a special token.

    def __init__(self, vocab):
        # Initializes the tokenizer with a vocabulary dictionary.

        # Store the original string-to-integer mapping
        self.str_to_int = vocab

        # Create a reverse integer-to-string mapping for decoding
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        # Converts a text string into a list of integer IDs.

        # 1. Split the text into tokens based on punctuation and whitespace
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        # 2. Remove any empty or whitespace-only strings from the list
        preprocessed = [item.strip() for item in preprocessed if item.strip()]

        # 3. Replace any token not in the vocabulary with the "<|unk|>" token
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        # 4. Convert all tokens in the list to their integer IDs
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        # Converts a list of integer IDs back into a text string.

        # 1. Look up each integer ID to get its string token, then join them with spaces
        text = " ".join([self.int_to_str[i] for i in ids])

        # 2. Clean up spacing around punctuation for better readability
        #    (e.g., converts "Hello , world ." to "Hello, world.")
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

Let's try to tokenize text with the modified tokenizer:

In [28]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [29]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [30]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 BytePair encoding

- GPT-2 used BytePair encoding (BPE) as its tokenizer
- it allows the model to break down words that aren't in its predefined vocabulary into smaller subword units or even individual characters, enabling it to handle out-of-vocabulary words
- For instance, if GPT-2's vocabulary doesn't have the word "unfamiliarword," it might tokenize it as ["unfam", "iliar", "word"] or some other subword breakdown, depending on its trained BPE merges
- The original BPE tokenizer can be found here: [https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- In this chapter, we are using the BPE tokenizer from OpenAI's open-source [tiktoken](https://github.com/openai/tiktoken) library, which implements its core algorithms in Rust to improve computational performance
- I created a notebook in the [./bytepair_encoder](../02_bonus_bytepair-encoder) that compares these two implementations side-by-side (tiktoken was about 5x faster on the sample text)

In [32]:
!pip install tiktoken

In [33]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.11.0


In [34]:
tokenizer = tiktoken.get_encoding("gpt2")

In [35]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [36]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPE tokenizers break down unknown words into subwords and individual characters:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="300px">

## 2.6 Data sampling with a sliding window

- We train LLMs to generate one word at a time, so we want to prepare the training data accordingly where the next word in a sequence represents the target to predict:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="400px">

In [38]:
# Open and read the entire content of the file again.
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# Use our tokenizer to encode the entire story into a list of integer IDs.
enc_text = tokenizer.encode(raw_text)

# Print the total number of tokens in the encoded text.
print(len(enc_text))

5145


- For each text chunk, we want the inputs and targets
- Since we want the model to predict the next word, the targets are the inputs shifted by one position to the right

In [39]:
enc_sample = enc_text[50:]

In [40]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- One by one, the prediction would look like as follows:

In [41]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [42]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- We will take care of the next-word prediction in a later chapter after we covered the attention mechanism
- For now, we implement a simple data loader that iterates over the input dataset and returns the inputs and targets shifted by one

- Install and import PyTorch (see Appendix A for installation tips)

In [43]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.8.0+cu126


- We use a sliding window approach, changing the position by +1:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">

- Create dataset and dataloader that extract chunks from the input text dataset

In [46]:
# Import the Dataset and DataLoader classes from PyTorch's data utilities
from torch.utils.data import Dataset, DataLoader

# Define a custom Dataset class for preparing GPT training data
class GPTDatasetV1(Dataset):
    # This method initializes the dataset object.
    def __init__(self, txt, tokenizer, max_length, stride):
        # Initialize empty lists to store the input and target sequences.
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire input text into a single list of integer IDs.
        # The `allowed_special` argument ensures that specific special tokens are processed.
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Check if the tokenized text is long enough to create at least one sequence.
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to create overlapping chunks from the tokenized text.
        # The loop iterates through the tokens with a step size of `stride`.
        for i in range(0, len(token_ids) - max_length, stride):
            # Extract the input chunk of size `max_length`.
            input_chunk = token_ids[i:i + max_length]
            # Extract the target chunk, which is the input chunk shifted one position to the right.
            target_chunk = token_ids[i + 1: i + max_length + 1]
            # Convert the input chunk to a PyTorch tensor and append it to the list.
            self.input_ids.append(torch.tensor(input_chunk))
            # Convert the target chunk to a PyTorch tensor and append it to the list.
            self.target_ids.append(torch.tensor(target_chunk))

    # This method returns the total number of samples (chunks) in the dataset.
    def __len__(self):
        return len(self.input_ids)

    # This method retrieves a single sample (an input-target pair) from the dataset at a given index.
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [47]:
# This function creates a data loader for training a GPT model.
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer using tiktoken's pre-built "gpt2" encoder.
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create an instance of our custom dataset.
    # It will tokenize the text and create overlapping input/target chunks.
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Wrap the dataset in a PyTorch DataLoader.
    # The DataLoader will handle batching, shuffling, and parallel data loading.
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,  # Number of samples per batch
        shuffle=shuffle,        # Randomize the order of samples each epoch
        drop_last=drop_last,    # Drop the last incomplete batch
        num_workers=num_workers # Number of subprocesses to use for data loading
    )

    # Return the fully configured dataloader.
    return dataloader

- Let's test the dataloader with a batch size of 1 for an LLM with a context size of 4:

In [48]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [49]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [50]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- An example using stride equal to the context length (here: 4) as shown below:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">

- We can also create batched outputs
- Note that we increase the stride here so that we don't have overlaps between the batches, since more overlap could lead to increased overfitting

In [51]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 Creating token embeddings

- The data is already almost ready for an LLM
- But lastly let us embed the tokens in a continuous vector representation using an embedding layer
- Usually, these embedding layers are part of the LLM itself and are updated (trained) during model training

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="400px">

- Suppose we have the following four input examples with input ids 2, 3, 5, and 1 (after tokenization):

In [52]:
input_ids = torch.tensor([2, 3, 5, 1])

- For the sake of simplicity, suppose we have a small vocabulary of only 6 words and we want to create embeddings of size 3:

In [54]:
# Define the total number of unique tokens in our vocabulary.
vocab_size = 6

# Define the size of the vector that will represent each token.
output_dim = 3

# Set the seed for PyTorch's random number generator to ensure reproducible results.
torch.manual_seed(123)

# Create an embedding layer.
# This layer acts as a lookup table, mapping each token's integer ID to a dense vector.
# It will have `vocab_size` rows and `output_dim` columns.
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- This would result in a 6x3 weight matrix:

In [55]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- For those who are familiar with one-hot encoding, the embedding layer approach above is essentially just a more efficient way of implementing one-hot encoding followed by matrix multiplication in a fully-connected layer, which is described in the supplementary code in [./embedding_vs_matmul](../03_bonus_embedding-vs-matmul)
- Because the embedding layer is just a more efficient implementation that is equivalent to the one-hot encoding and matrix-multiplication approach it can be seen as a neural network layer that can be optimized via backpropagation

- To convert a token with id 3 into a 3-dimensional vector, we do the following:

In [56]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- Note that the above is the 4th row in the `embedding_layer` weight matrix
- To embed all four `input_ids` values above, we do

In [57]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- An embedding layer is essentially a look-up operation:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

## 2.8 Encoding word positions

- Embedding layer convert IDs into identical vector representations regardless of where they are located in the input sequence:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="400px">

- Positional embeddings are combined with the token embedding vector to form the input embeddings for a large language model:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

- The BytePair encoder has a vocabulary size of 50,257:
- Suppose we want to encode the input tokens into a 256-dimensional vector representation:

In [59]:
# Define the vocabulary size, corresponding to the number of unique tokens.
# 50257 is the vocabulary size used by the GPT-2 model.
vocab_size = 50257

# Define the dimensionality of the embedding vectors.
# Each token will be represented by a vector of this size.
output_dim = 256

# Create the token embedding layer.
# This layer will hold the vector representations for all tokens in the vocabulary.
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- If we sample data from the dataloader, we embed the tokens in each batch into a 256-dimensional vector
- If we have a batch size of 8 with 4 tokens each, this results in a 8 x 4 x 256 tensor:

In [60]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [61]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [62]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(token_embeddings)

torch.Size([8, 4, 256])


- GPT-2 uses absolute position embeddings, so we just create another embedding layer:

In [63]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# uncomment & execute the following line to see how the embedding layer weights look like
# print(pos_embedding_layer.weight)

In [64]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(pos_embeddings)

torch.Size([4, 256])


- To create the input embeddings used in an LLM, we simply add the token and the positional embeddings:

In [65]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(input_embeddings)

torch.Size([8, 4, 256])


- In the initial phase of the input processing workflow, the input text is segmented into separate tokens
- Following this segmentation, these tokens are transformed into token IDs based on a predefined vocabulary:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">

# Summary and takeaways

See the [./dataloader.ipynb](./dataloader.ipynb) code notebook, which is a concise version of the data loader that we implemented in this chapter and will need for training the GPT model in upcoming chapters.

See [./exercise-solutions.ipynb](./exercise-solutions.ipynb) for the exercise solutions.

See the [Byte Pair Encoding (BPE) Tokenizer From Scratch](../02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb) notebook if you are interested in learning how the GPT-2 tokenizer can be implemented and trained from scratch.